### import Modules

In [1]:
import pandas as pd
import numpy as np
import time
import os
import copy
import json

# importing plotting lib
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image

# importing all torch lib
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import models
from torchvision import transforms
from tqdm import tqdm
# for augmentaiton
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

# for ignoring warnings
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

### Load the Data

In [2]:
BASE_DIR = '../input/cassava-leaf-disease-classification/'

In [3]:
train = pd.read_csv(BASE_DIR + 'train.csv')
train.head()

FileNotFoundError: [Errno 2] No such file or directory: '../input/cassava-leaf-disease-classification/train.csv'

In [ ]:
## loading mapping for target label

with open(BASE_DIR + "label_num_to_disease_map.json") as f:
    mapping = json.loads(f.read())
    mapping = {int(k):v for k, v in mapping.items()}
mapping

In [ ]:
train["label_name"] = train['label'].map(mapping)
train.head()

### Exploratory Data Analysis

In [ ]:
def plot_images(class_id, label, total_images = 6):
    # get image ids correcsponding to the target class id
    plot_list = train[train['label'] == class_id].sample(total_images)['image_id'].tolist()
    
    labels = [label for i in range(total_images)]
    size = int(np.sqrt(total_images))
    if size*size < total_images:
        size += 1
    plt.figure(figsize =(15, 15))
    
    #plot the image in subplot
    for index, (image_id, label) in enumerate(zip(plot_list, labels)):
        plt.subplot(size, size, index + 1)
        image = Image.open(str(BASE_DIR + "train_images/" +image_id))
        plt.imshow(image)
        plt.title(label, fontsize = 14)
        plt.axis("off")
        
    plt.show()
        
        
    

In [ ]:
plot_images(0, mapping[0], 6)

In [ ]:
# class distribution
sns.countplot(train["label"])

#### Obs: The label distribution is skewed. Hence We should use AUC for metric and Stratified KFold for data spliting

### configuration and Utility Functions

In [ ]:
# All constant config

DIM = (256, 256)
WIDTH, HEIGHT = DIM
NUM_CLASSES = 5
NUM_WORKERS = 24
TRAIN_BATCH_SIZE = 128
TEST_BATCH_SIZE = 128
SEED = 4
DEVICE  = 'cuda'
MEAN = (0.485,0.456, 0.406)
STD = (0.229, 0.224, 0.225)
LR = 0.001

### Augmentations

In [ ]:
def get_test_transform(value = 'val'):
    if value == 'train':
        return A.Compose([
            A.Resize(WIDTH, HEIGHT),
            A.HorizontalFlip(p = 0.5),
            A.Rotate(limit = (-90, 90)),
            A.VerticalFlip(p = 0.5),
            A.Normalize(MEAN, STD, max_pixel_value = 255.0, always_apply = True),
            ToTensorV2(p=1.0) # returning tensor for all images
        ])
    elif value == 'val':
        return A.Compose([
            A.Resize(WIDTH, HEIGHT),
            A.Normalize(MEAN, STD, max_pixel_value = 255.0, always_apply = True),
            ToTensorV2(p=1.0)
        ])
        

### Dataset Loader Class

In [ ]:
class CassavaDataset(Dataset):
    def __init__(self, image_ids, labels, dim = None, aug = None, folder = 'train_images'):
        super().__init__()
        self.image_ids = image_ids
        self.labels = labels
        self.dim = dim
        self.aug = aug
        self.folder = folder
        
    def __len__(self):
        return len(self.image_ids)
    
    def __getitem__(self, index):
        image = self.image_ids[index]
        img = Image.open(os.path.join(BASE_DIR, self.folder, image))
        
        if(self.dim):
            img = img.resize(self.dim)
            
        img = np.array(img)
        
        if self.aug is not None:
            augmented = self.aug(image = img)
            img = augmented["image"]
            
        label = torch.tensor(self.labels[index], dtype = torch.long)
        return img, label
            
        
        

### Create Folds

In [ ]:
from sklearn.model_selection import StratifiedKFold
kf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 20)
train['kfold']  = -1
for fold, (train_idx, val_idx) in enumerate(kf.split(X = train['image_id'], y = train['label'])):
    train.loc[val_idx, 'kfold'] = fold
train.head()

### Use Pretrained Model (Transfer learning)

In [ ]:
def getmodel():
    net = models.efficientnet_b0(pretrained = True)
    
    ## freeze all the layers in the network
    for param in net.parameters():
        param.requires_grad = False
        
    num_ftrs =1280 
    # create last few layers
    net.classifier[1] = nn.Sequential(
        nn.Linear(num_ftrs, 256),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(256, NUM_CLASSES),
        nn.LogSoftmax(dim = 1)
    )
    # use gpu if any
    net = net.to(device = DEVICE)
    return net

In [ ]:
model = getmodel()

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = LR)

In [ ]:
# find total parameters in the model
total_params = sum(p.numel() for p in model.parameters())
print(f"total_params: {total_params:,}")
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"train_params: {trainable_params:,}")

### Steps for Training and Validaiton

In [ ]:


def train_model(model, dataloaders, criterion, optimizer, num_epochs=5):
    # set starting time
    start_time = time.time()
    
    val_acc_history = []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in tqdm(range(num_epochs)):
        print(f'Epoch {epoch}/{num_epochs-1}')
        print('-'*15)
        
        # each epoch have training and validation phase
        for phase in ['train', 'val']:
            # set mode for model
            if phase == 'train':
                model.train() # set model to training mode
            else:
                model.eval() # set model to evaluate mode
                
            running_loss = 0.0
            running_corrects = 0
            fin_out = []
            
            # iterate over data
            for inputs, labels in dataloaders[phase]:
                # move data to corresponding hardware
                inputs = inputs.to(DEVICE)
                labels = labels.to(DEVICE)
                
                # reset (or) zero the parameter gradients
                optimizer.zero_grad()
                
                # training (or) validation process
                with torch.set_grad_enabled(phase=='train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    
                    _, preds = torch.max(outputs, 1)
                    
                    # back propagation in the network
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                        
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            # calculate loss and accuarcy for the epoch
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            
            # print loss and acc for training & validation
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            
            # update the best weights
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)
                
        print()
    end_time = time.time() - start_time
    
    print('Training completes in {:.0f}m {:.0f}s'.format(end_time // 60, end_time % 60))
    print('Best Val Acc: {:.4f}'.format(best_acc))
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history



In [ ]:
## train the model
for fold in tqdm(range(5)):
    print(f"Fold: {fold}")
    # create train data and val data
    train_data = train[train['kfold'] != fold]
    val_data = train[train['kfold'] == fold]
    
    # create trainset, trainloader
    train_dataset = CassavaDataset(
    image_ids = train_data['image_id'].values, 
    labels = train_data['label'].values,
    aug = get_test_transform('train'),
    dim = DIM)

    train_loader  = DataLoader(
        train_dataset, 
        batch_size = TRAIN_BATCH_SIZE, 
        shuffle = False, 
        num_workers = NUM_WORKERS)
    
    #create the valset and valloader
    val_dataset = CassavaDataset(
    image_ids = val_data['image_id'].values, 
    labels = val_data['label'].values,
    aug = get_test_transform('val'),
    dim = DIM)

    val_loader  = DataLoader(
        val_dataset, 
        batch_size = TRAIN_BATCH_SIZE, 
        shuffle = False, 
        num_workers = NUM_WORKERS)
    
    loader = {"train":train_loader, "val":val_loader}
    
    model, accuracy = train_model(model = model, 
                                  dataloaders = loader, 
                                  criterion = criterion, 
                                  optimizer = optimizer, 
                                  num_epochs = 5)
    torch.save(model, f'/kaggle/working/best_model_{fold}.h5')
    torch.save(model.state_dict(), f'/kaggle/working/best_model_weights_{fold}')

In [ ]:
torch.cuda.empty_cache()